## Python Decorators
Python decorators is a function that takes in another funtion and entend it without explicitly modifying it.

**Use Case :**    
They are often used for logging, enforcing access control, instrumentation, caching, and more.

**Definition :**    
A function what returns a wrapper function

In [1]:
def decorator_func(my_func):
    def wrapper_func(*args, **kwargs):
        print(f"Wrapper Function is called before {my_func.__name__}")
        return my_func(*args, **kwargs)

    return wrapper_func

@decorator_func
def display():
    print("Decorator display")

display()

Wrapper Function is called before display
Decorator display


*use @decorator_name or by this way (both are same)*

```python
decorator_display = decorator_func(display)
decorator_display()
```
The decorator function returns a callable function which can be called

### Decorators with multiple args, kwargs

In [2]:
# Decorators with multiple args, kwargs
@decorator_func
def display_info(name, age):
    print("Decorator display information")
    print(f"Name: {name}, Age: {age}")

display_info("Vishnu", 10)


Wrapper Function is called before display_info
Decorator display information
Name: Vishnu, Age: 10


### Decorators with class

In [3]:
class decorator_class:

    # instantiating the function in our class
    def __init__(self, my_func):
        self.my_func = my_func

    # wrapper function
    def __call__(self, *args, **kwargs):
        print(f"Call method is called before {self.my_func.__name__}")
        return self.my_func(*args, **kwargs)

@decorator_class
def display():
    print("Decorator display")

display()        

Call method is called before display
Decorator display


### Use case 1: Logging

Keep track of how many times a function is executed and what arguments are passed to it

In [5]:
def my_logger(my_func):

    import logging
    logging.basicConfig(filename = f"{my_func.__name__}.log", level=logging.INFO)

    def wrapper_func(*args, **kwargs):
        logging.info(f'{my_func.__name__} run with arguments: {args} and keyword arguments: {kwargs}')

        return my_func(*args, **kwargs)
    return wrapper_func

@my_logger
def display_info(name, age):
    print("Decorator display information")
    print(f"Name: {name}, Age: {age}")

display_info("Vishnu", age=10)

Decorator display information
Name: Vishnu, Age: 10


### Use case 2: Timer

calculating the time taken for a function to be executed

In [6]:
def my_timer(func):
    import time

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = func(*args, **kwargs)
        t2 = time.time() - t1
        print(f"{func.__name__} is executed in {t2} seconds")
        return result

    return wrapper

import time
@my_timer
def display_info(name, age):
    time.sleep(2)
    print("Decorator display information")
    print(f"Name: {name}, Age: {age}")

display_info("Vishnu", 20)

Decorator display information
Name: Vishnu, Age: 20
display_info is executed in 2.002424716949463 seconds


### Multiple decorators for a single function

In [7]:
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(2)
    print("Decorator display information")
    print(f"Name: {name}, Age: {age}")

display_info("Vishnu", 15)

Decorator display information
Name: Vishnu, Age: 15
display_info is executed in 2.0015182495117188 seconds


This works fine but when we stack decorators it will be executed based on an order

``` python
# my_timer -> my_logger
result = my_logger(my_timer(function))
```

Here my_logger(wrapper) will be the output of my_timer(function) so wrapper will be the function passed to my_logger and it will be logged
``` markdown
INFO:root:wrapper run with arguments: ('Vishnu', 15) and keyword arguments: {}
*will be logged `wrapper` run with args...*
```

To execute decorators in the correct order we can use `functools.wraps()`

In [9]:
from functools import wraps

def my_logger(func):

    import logging
    logging.basicConfig(filename = f"{func.__name__}.log", level=logging.INFO)

    @wraps(func)
    def wrapper(*args, **kwargs):
        logging.info(f'{func.__name__} run with arguments: {args} and keyword arguments: {kwargs}')

        return func(*args, **kwargs)
    return wrapper

def my_timer(func):
    import time

    @wraps(func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = func(*args, **kwargs)
        t2 = time.time() - t1
        print(f"{func.__name__} is executed in {t2} seconds")
        return result

    return wrapper


@my_logger
@my_timer
def display_info(name, age):
    time.sleep(2)
    print("Decorator display information")
    print(f"Name: {name}, Age: {age}")

display_info("Tom", 100)


Decorator display information
Name: Tom, Age: 100
display_info is executed in 2.0023951530456543 seconds


### Decorators with Arguments

In [12]:
def decorator_args(arguments):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            print(f"{arguments}{func.__name__} is executed")
            return func()
        return wrapper
    return my_decorator


@decorator_args("TEST: ")
def display():
    print("Decorator display")

display()

TEST: display is executed
Decorator display


Wrap the decorator function inside another function to accept arguments